#### As usual, there are two method
- [Core method](https://docs.sqlalchemy.org/en/20/tutorial/data_insert.html#tutorial-core-insert)
- [ORM method](https://docs.sqlalchemy.org/en/20/tutorial/orm_data_manipulation.html#tutorial-orm-data-manipulation)

#### here illustrated orm method

#### refresher/prerequisite to the orm method
- [Executing with an ORM Session - introduces how to make an ORM Session object](https://docs.sqlalchemy.org/en/20/tutorial/orm_data_manipulation.html#tutorial-orm-data-manipulation)
- [Using ORM Declarative Forms to Define Table Metadata - where we set up our ORM mappings of the User and Address entities](https://docs.sqlalchemy.org/en/20/tutorial/metadata.html#tutorial-orm-table-metadata)
- [Selecting ORM Entities and Columns - a few examples on how to run SELECT statements for entities like User](https://docs.sqlalchemy.org/en/20/tutorial/data_select.html#tutorial-selecting-orm-entities)

In [4]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import ForeignKey
from sqlalchemy.orm import Session
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

class User(Base):
    
    __tablename__ = "user_account"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    
    __tablename__ = "address"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"
    
from sqlalchemy import create_engine

template_engine = "postgresql+psycopg://{db_username}:{db_password}@{db_host}:{db_port}"


engine_config = template_engine.format(
        db_username="a",
        db_password="v",
        db_host="c.cosqamqjez6h.ap-northeast-2.rds.amazonaws.com",
        db_port="5432",
        db_name="")



engine = create_engine(engine_config,
    echo=True
) 


## select

In [7]:
from sqlalchemy import select

with Session(engine) as session:

    squidwardclone = session.get(User, 10)
    
    print(squidwardclone)

    #select with filter
    ehkrabsclone = session.execute(select(User).filter_by(name="ehkrabs"))
    
    for item in ehkrabsclone:
        print(item)

2023-03-06 21:45:01,260 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 21:45:01,263 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = %(pk_1)s::INTEGER
2023-03-06 21:45:01,264 INFO sqlalchemy.engine.Engine [cached since 445.8s ago] {'pk_1': 10}
None
2023-03-06 21:45:01,330 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = %(name_1)s::VARCHAR(30)
2023-03-06 21:45:01,331 INFO sqlalchemy.engine.Engine [cached since 445.9s ago] {'name_1': 'ehkrabs'}
(User(id=15, name='ehkrabs', fullname='Eugene H. Krabs'),)
(User(id=17, name='ehkrabs', fullname='Eugene H. Krabs'),)
2023-03-06 21:45:01,339 INFO sqlalchemy.engine.Engine ROLLBACK


### Update

In [11]:
from sqlalchemy import update
from sqlalchemy.orm import Session

with Session(engine) as session:

    session.execute(update(User)
        .where(User.id == 15)
        .values(fullname="Pegasus Extraordinaire")
    )
    
    session.flush()
    
    session.commit()

    updateddata = session.execute(select(User).filter_by(id=15)).first()
    print(updateddata)
    

2023-03-06 21:46:00,758 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 21:46:00,760 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=%(fullname)s::VARCHAR WHERE user_account.id = %(id_1)s::INTEGER
2023-03-06 21:46:00,761 INFO sqlalchemy.engine.Engine [cached since 26.83s ago] {'fullname': 'Pegasus Extraordinaire', 'id_1': 15}
2023-03-06 21:46:00,779 INFO sqlalchemy.engine.Engine COMMIT
2023-03-06 21:46:00,790 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 21:46:00,791 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = %(id_1)s::INTEGER
2023-03-06 21:46:00,792 INFO sqlalchemy.engine.Engine [cached since 26.67s ago] {'id_1': 15}
(User(id=15, name='ehkrabs', fullname='Pegasus Extraordinaire'),)
2023-03-06 21:46:00,819 INFO sqlalchemy.engine.Engine ROLLBACK
